### Imports

In [ ]:
import glob, scipy.io, matplotlib.pyplot as plt, numpy as np, os, seaborn as sns, pandas as pd, \
    matplotlib.collections as mc, regex as re, os, matplotlib.colors as mplcol, shapely.geometry as geom
from scipy.stats import gaussian_kde
from tqdm import tqdm_notebook as tqdm

### Load Data

In [ ]:
# Enumerate all manual annotations
fnamesAll = glob.glob('Z:/behavior/*/croprot/*.manual_annotation.labels.mat')
fnamesAll = [x for x in fnamesAll if 'RIG' not in x]
len(fnamesAll)

In [ ]:
#@nb.njit(nogil=True)
def applyRotationAlongAxis(R, X):
    """
    This helper function applies a rotation matrix to every <X, Y> position tuple in a Nx2 matrix.
    Note: Numba JIT leads to a ~6-fold speed improvement.
    """
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            X[i, j, 0:2] = R[:, :, i] @ X[i, j, 0:2]

In [ ]:
def loadData(fnameGT):
    fnameGTf = fnameGT.replace('.labels.mat','.frames_used.npy')
    fnameDLC = fnameGT.replace('.manual_annotation.labels.mat','_dlc.npy')
    fnameLeap= fnameGT.replace('.manual_annotation.labels.mat','_leap.npy')
    
    if os.path.exists(fnameDLC.replace('.npy', '.2.npy')):
        fnameDLC = fnameDLC.replace('.npy', '.2.npy')
    if os.path.exists(fnameLeap.replace('.npy', '.2.npy')):
        fnameLeap = fnameLeap.replace('.npy', '.2.npy')
    
    try:
        dataGT     = np.moveaxis(scipy.io.loadmat(fnameGT)['positions'], 2, 0)
    except Exception as e:
        print(e)
        return np.array([]), None, None, None, None, None, None
    framesUsed = np.load(fnameGTf)
    
    N = int(np.memmap(fnameDLC, dtype=np.float32).size / (26 * 3))
    dataLeapAll = np.memmap(fnameLeap, shape=(N, 3, 26), mode='r', dtype=np.float32)
    dataDLCAll  = np.memmap(fnameDLC , shape=(N, 3, 26), mode='r', dtype=np.float32)
    
    dataLeap = np.moveaxis(dataLeapAll[framesUsed,:,:].copy(), 2, 1)[:,:,0:2]
    dataDLC  = np.moveaxis(dataDLCAll [framesUsed,:,:].copy(), 2, 1)[:,:,0:2]
    
    # Obtain the subsetted dataset of actually annotated frames
    framesAnnotated = np.any(np.any(~np.isnan(dataGT), axis=2), axis=1)

    dataGT   = dataGT  [framesAnnotated] - 100
    dataLeap = dataLeap[framesAnnotated] - 100
    dataDLC  = dataDLC [framesAnnotated] - 100
    
    # Set missing coordinates to NaN: GT
    dataGT[np.linalg.norm(dataGT, axis=2) > 90,:] = np.nan
    
    # Load corrected frames
    dataDLCcorr = None
    dataLeapcorr= None
    
    for method in ['leap', 'dlc']:
        pos = np.memmap(fnameGT.replace('.manual_annotation.labels.mat','_mat.npy'), 
                        dtype=np.double, shape=(dataLeapAll.shape[0], 4), mode='r')[framesUsed, :]

        corr = np.load(fnameGT.replace('.manual_annotation.labels.mat',
            '_{}_abs_filt_interp.npy'.format(method)))[framesUsed,:,0:2]
        corr -= np.repeat(pos[:,1:3][:,np.newaxis,:], 26, axis=1)

        # Only keep annotated frames
        pos  = pos [framesAnnotated]
        corr = corr[framesAnnotated].astype(np.float64)
        
        theta = -pos[:,3] * np.pi / 180.0
        c, s = np.cos(theta), np.sin(theta)
        R = np.array(((c, -s), (s, c)), dtype=np.double)
        applyRotationAlongAxis(R, corr)
        
        if method == 'leap':
            # For now, don't compare inferred points that are missing in LEAP/DLC
            corr[np.isnan(dataLeap)] = np.nan
            dataLeapcorr = corr
        elif method == 'dlc':
            dataDLCcorr = corr

    return dataGT, dataLeap, dataDLC, dataLeapcorr, dataDLCcorr, dataLeapAll, dataDLCAll

In [ ]:
def getErrors(fname, corrected = False):
    dataGT, dataLeap, dataDLC, dataLeapcorr, \
        dataDLCcorr, dataLeapAll, dataDLCAll = loadData(fname)
    
    if dataGT.size == 0:
        return None
    
    # Compute errors
    if corrected:
        errLeap = dataLeapcorr - dataGT
        errDLC  = dataDLCcorr  - dataGT
    else:
        errLeap = dataLeap - dataGT
        errDLC  = dataDLC  - dataGT

    errLeapsk = errLeap + np.repeat(skeleton[np.newaxis, :, :], errLeap.shape[0], axis=0)
    errDLCsk  = errDLC  + np.repeat(skeleton[np.newaxis, :, :], errLeap.shape[0], axis=0)
    
    return errLeap, errDLC, errLeapsk, errDLCsk, dataGT

### Plotting function

In [ ]:
def plotFrame(d, fidx, ax, jointcolor='#000000', segmentcolor='#000000'):
    # Draw body joints
    for i in range(26):
        _x = d[fidx, i,0]
        _y = d[fidx, i,1]
        if not np.isnan(_x) and not np.isnan(_y):
            pass
            #ax.scatter(_x, _y, c=jointcolor)
            #ax.text(_x, _y, i, fontsize=12, color=jointcolor)
    
    def _plot(x, y, c):
        if not np.isnan(x[0]) and \
            not np.isnan(x[1]) and \
            not np.isnan(y[0]) and \
            not np.isnan(y[1]):
            ax.plot(x, y, c)

    # Draw body segments
    def _p(i, j): return [
        (d[fidx, i, 0], d[fidx, j, 0]), 
        (d[fidx, i, 1], d[fidx, j, 1]), segmentcolor]

    _plot( *_p(14,18) ); _plot( *_p(18,22) )
    _plot( *_p( 2, 6) ); _plot( *_p( 6,10) )

    _plot( *_p(15,19) ); _plot( *_p(19,23) )
    _plot( *_p( 3, 7) ); _plot( *_p( 7,11) )

    _plot( *_p(16,20) ); _plot( *_p(20,24) )
    _plot( *_p( 4, 8) ); _plot( *_p( 8,12) )

    _plot( *_p(17,21) ); _plot( *_p(21,25) )
    _plot( *_p( 5, 9) ); _plot( *_p( 9,13) )
    
    # Set bounding box for display
    plt.xlim(-75, 75)
    plt.ylim(-75, 75)

### Create display skeleton

In [ ]:
skeleton = np.array([
    [ 12, 0, 0], # 0
    [-12, 0, 0], # 1
    [ 10, 10, 0], # 2
    [5, 15, 0], # 3
    [-5, 20, 0], # 4
    [-10, 15, 0], # 5
    [20,20, 0], # 6
    [10, 35, 0], # 7
    [-5, 40, 0], # 8
    [-20,30, 0], # 9
    [40,30, 0], # 10
    [0, 60, 0], # 11
    [-15, 60, 0], # 12
    [-40,30, 0], # 13
    [10, -10, 0], # 14
    [5, -15, 0], # 15
    [-5, -20, 0], # 16
    [-10, -15, 0], # 17
    [20,-20, 0], # 18
    [10, -40, 0], # 19
    [-5, -40, 0], # 20
    [-20,-30, 0], # 21
    [40,-30, 0], # 22
    [0, -60, 0], # 23
    [-15, -60, 0], # 24
    [-40,-30, 0]  # 25
], dtype=np.float32)[:,0:2]

skeleton *= np.array([2, 1])

fig, ax = plt.subplots(1, 1, figsize=(8,8))
plotFrame(np.array([skeleton,]), 0, ax)
plt.show(fig)

### Color Palette

In [ ]:
# Define color scheme
COLORS = [
    (0, 0, 0),
    (230, 159, 0),
    (0, 158, 115),
    (0, 114, 178),
    (204, 121, 167)
]
COLORS = [mplcol.rgb2hex(np.array(x) / 255.0) for x in COLORS]

### Count datapoints used

In [ ]:
errs = [getErrors(x, corrected=True) for x in tqdm(fnamesAll)]

In [ ]:
np.sum([x[0].shape[0] for x in errs if x is not None])

### Plot

In [ ]:
def plotErrors(ax, method = 'dlc'):
    # Check method parameter
    if method not in ['dlc', 'leap']:
        raise Exception('Method should be either dlc or leap')
    
    # Compute errors
    errs = [getErrors(x, corrected=True) for x in tqdm(fnamesAll)]
    
    # Superimpose errors onto skeleton
    errsDLC = np.vstack([x[1 if method == 'dlc' else 0] for x in errs if x is not None])
    errsDLC = errsDLC + np.array(skeleton)[np.newaxis, :, :]
    
    # Monkey-patch Seaborn to enable extraction of contour
    def contourMonkeyPatch(contourPrev, out):
        def _f(*k, **kw):
            r = contourPrev(*k, **kw)
            out.append(r)
            return r
        return _f
    
    for i in range(26):
        # Monkey-Patch contour function to save the contour coordinates to the 'arrContours' list
        arrContours = []
        contourPrev = ax.contour
        ax.contour = contourMonkeyPatch(contourPrev, arrContours)
        # Plot 95% contour
        sns.kdeplot(x=errsDLC[:,i,0], ax=ax, y=errsDLC[:,i,1], levels=[.05,], color=COLORS[0], zorder=10,
                    label=None if i != 0 else '95th percentile')
        ax.contour = contourPrev
        # Determine outlier points
        xyOutlier = np.logical_not(np.any(np.array([[geom.Polygon(seg).contains(geom.Point(p)) for p in errsDLC[:,i,:]] for \
            seg in arrContours[0].allsegs[0] if len(seg) > 0]), axis=0))
        xyOutliers = errsDLC[xyOutlier, i, :]
        # Plot outliers
        if i not in [0, 1, 2, 3, 4, 5, 14, 15, 16, 17]:
            ax.scatter(xyOutliers[:,0], xyOutliers[:,1], color='red', s=10)
            for k in range(xyOutliers.shape[0]):
                ax.plot([skeleton[i,0], xyOutliers[k,0]], 
                        [skeleton[i,1], xyOutliers[k,1]], color='gray')
        # Plot remaining contours
        sns.kdeplot(x=errsDLC[:,i,0], ax=ax, y=errsDLC[:,i,1], levels=[.50,], color=COLORS[1], zorder=10,
                    label=None if i != 0 else '50th percentile')
        sns.kdeplot(x=errsDLC[:,i,0], ax=ax, y=errsDLC[:,i,1], levels=[.25,], color=COLORS[2], zorder=10,
                    label=None if i != 0 else '75th percentile')

    ax.set_axis_off()

    plotFrame(np.array([skeleton,]), 0, ax)
    ax.set_xlim(-120, 120)
    ax.set_ylim(-100, 100)
    
    ax.legend()
    
    ax.set_title(method)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 16))

plotErrors(ax[0], 'dlc')
plotErrors(ax[1], 'leap')

fig.savefig('C:/Users/acorver/Desktop/paper-figures/fig1d.pdf', dpi=1000)

### Supplementary Figure

In [ ]:
errsUncorr = [getErrors(x, corrected=False) for x in tqdm(fnamesAll)]

In [ ]:
MAXERROR = 30

In [ ]:
data = []
for method in ['dlc', 'leap']:
    for err in errsUncorr:
        if err is not None:
            _err = err[1 if method == 'dlc' else 0]
            for i in range(_err.shape[0]):
                for limb in range(_err.shape[1]):
                    data.append((method, limb, min(MAXERROR, np.linalg.norm(_err[i, limb, :]))))
                    
data = pd.DataFrame(data, columns=['method', 'limb', 'error'])

In [ ]:
def getKDE(data, limb, method):
    v = data.error[(data.limb==limb)&(data.method==method)]
    v = v[~np.isnan(v)]
    kdex = np.linspace(0, MAXERROR, 100)
    kde = gaussian_kde(v)
    kde.set_bandwidth(0.1)
    return kdex, kde.evaluate(kdex)

In [ ]:
fig, ax = plt.subplots(5, 6, figsize=(12, 9))
subplotPositions = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29]
subplotEmpty = list(set(list(range(30))) - set(subplotPositions))
for limb in range(30):    
    if limb < 26:
        icol = subplotPositions[limb]%6
        irow = int(subplotPositions[limb]/6)

        kdex1, kde1 = getKDE(data, limb, 'dlc')
        kdex2, kde2 = getKDE(data, limb, 'leap')

        ax[irow][icol].plot(kdex1, kde1, color='red', label='DeepLabCut' if limb == 0 else None)
        ax[irow][icol].plot(kdex2, kde2, color='blue', label='LEAP' if limb == 0 else None)

        ax[irow][icol].set_title(limb)
        ax[irow][icol].set_xlim(0, MAXERROR+1)
        ax[irow][icol].set_ylim(0, 0.5)

        ax[irow][icol].get_xaxis().set_visible(irow == 4)
        ax[irow][icol].get_yaxis().set_visible(icol == 0)
        
        ax[irow][icol].set_yticks(np.linspace(0, 0.5, 6))
    else:
        icol = subplotEmpty[limb-26]%6
        irow = int(subplotEmpty[limb-26]/6)
        fig.delaxes(ax[irow][icol])
        
fig.legend()
fig.tight_layout()

fig.savefig('C:/Users/acorver/Desktop/paper-figures/Fig_Suppl_3C.pdf')

### A few statistics

In [ ]:
data.error[(data.method=='leap')&(data.limb.isin([0, 1, 6, 10, 18, 22, 9, 13, 21, 25]))].mean()

In [ ]:
data.error[(data.method=='dlc')&(data.limb.isin([0, 1, 6, 10, 18, 22, 9, 13, 21, 25]))].mean()

In [ ]:
data.error[(data.method=='leap')&(data.limb.isin([0, 1, 6, 10, 18, 22, 9, 13, 21, 25]))].median()

In [ ]:
data.error[(data.method=='dlc')&(data.limb.isin([0, 1, 6, 10, 18, 22, 9, 13, 21, 25]))].median()

In [ ]:
data.error[(data.method=='leap')&(data.limb.isin([0, 1, 6, 10, 18, 22, 9, 13, 21, 25]))&(data.error <= 25)].mean()

In [ ]:
data.error[(data.method=='dlc')&(data.limb.isin([0, 1, 6, 10, 18, 22, 9, 13, 21, 25]))&(data.error <= 25)].mean()

In [ ]:
(data.error[(data.method=='leap')&(data.limb.isin([0, 1, 6, 10, 18, 22, 9, 13, 21, 25]))] > 25).mean()

In [ ]:
(data.error[(data.method=='dlc')&(data.limb.isin([0, 1, 6, 10, 18, 22, 9, 13, 21, 25]))] > 25).mean()